# General workflow
This workflow is a starting point to demonstrate the functionality of the containers and to detect and classify difficult cases

In [1]:
import numpy as np
import pandas as pd
import skimage
import Images
import ModelUNet_v2 #version 2 learns areas and boundaries of nuclei
import matplotlib.pyplot as plt
import tqdm
import copy
import scipy
import plotly #plan to do interactive plots

/home/lienhard/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#some definitions
debug=False #perform on a subset --> faster
model_shape=(256,256,3) #works with shape (2^x, 2^x) only?
model_name='unet_{}x{}_xent_v2'.format(model_shape[0],model_shape[1])
if debug:
    model_name += '_debug'

print('the model will be called \''+model_name+'\'')


the model will be called 'unet_256x256_xent_v2'


## Training
* load training data
* train the model (if not yet done)
* 

In [12]:
train=Images.Images("../input/stage1_train")
if debug:
    #subselect, to make it faster 
    train=train.subset(idx=range(20))
    #todo: overload [] to get this

#set aside 10% for validation
val=train.subset(np.arange(train.n()*.9, train.n()))
train=train.subset(np.arange(train.n()*.9,))

#load the image files (in original size)    
train.load_images()
train.load_masks()
train.features.head()    
plt.figure(figsize=(18, 12), dpi= 80, facecolor='w', edgecolor='k')

train.show_image()

Empty DataFrame
Columns: []
Index: [180]


<Figure size 1440x960 with 0 Axes>

In [4]:
# initialize and train the model 
# to detect unlabled masks (e.g. the forground vs background)

model=ModelUNet_v2.ModelUNet(name=model_name, shape=model_shape)
#question: should the model design be adapted according to image dimensions?
if not model.trained:
    model.fit_model(train)


found area model file unet_256x256_xent_v2_area.h5
found boundary model file unet_256x256_xent_v2_boundary.h5


In [ ]:

#add_pred ruft models.predict auf, das folgendes macht:
# 1) area prediction (wie im beispiel von dem wir geklaut haben)
# 2) boundaries prediction (tolle neue innovation von uns)
# 3) beides nutzen um die labels zu asignen
# 4) rescalen zur originalgröße


#add labled predictions to container
train.add_predictions(model)
# this adds iou scores to train.features and train.nuc_features
# scores berechnen dautert ca 1 sec pro bild... ich berechne ne menge scores

print( train.features.drop(['ids'], axis=1).head() ) 
print("expected LB score(train): {}".format(np.mean(train.features['iou_score'])))
fig=plt.figure(figsize=(18, 12), dpi= 80, facecolor='w', edgecolor='k')

train.show_image()

In [ ]:
image_id=1
plt.figure(figsize=(18, 12), dpi= 80, facecolor='w', edgecolor='k')
train.show_image(image_id)
#print(train.nuc_features.head())

train.nuc_features.loc[train.nuc_features['img_id']==image_id,:]

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(18, 12), dpi= 80, facecolor='w', edgecolor='k')
train.show_image()

In [13]:
#same on validation data
val.load_images()
val.load_masks()
_val=copy.deepcopy(val)
val.add_predictions(model,boundary_height=0.05)
val.features.drop(['ids'], axis=1).head()
print("expected LB score(val): {}".format(np.mean(val.features['iou_score'])))


100%|██████████| 67/67 [00:08<00:00,  7.58it/s]


predicting area...
predicting boundaries...


  0%|          | 0/67 [00:00<?, ?it/s]/home/lienhard/.local/lib/python3.6/site-packages/skimage/morphology/extrema.py:111: FutureWarning:

Conversion of the second argument of issubdtype from `'half'` to `np.floating` is deprecated. In future, it will be treated as `np.float16 == np.dtype('half').type`.

/home/lienhard/.local/lib/python3.6/site-packages/skimage/segmentation/_join.py:118: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.

  4%|▍         | 3/67 [00:00<00:02, 23.69it/s]

labeling predictions...


  0%|          | 0/67 [00:00<?, ?it/s]

computing scores... 


100%|██████████| 67/67 [00:15<00:00,  4.19it/s]

expected LB score(val): 0.46290887005073494


In [15]:
#find optimum
mean_score=[]
for h in np.arange(0.01,.1,.01):
    _v=copy.deepcopy(_val)
    _v.add_predictions(model,boundary_height=h)
    score=np.mean(_v.features['iou_score'])
    print("expected LB score({}): {}".format(h, score))
    mean_score.append(score)
#optimal hight ~0.06

predicting area...
predicting boundaries...


  0%|          | 0/67 [00:00<?, ?it/s]/home/lienhard/.local/lib/python3.6/site-packages/skimage/morphology/extrema.py:111: FutureWarning:

Conversion of the second argument of issubdtype from `'half'` to `np.floating` is deprecated. In future, it will be treated as `np.float16 == np.dtype('half').type`.

/home/lienhard/.local/lib/python3.6/site-packages/skimage/segmentation/_join.py:118: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.

  4%|▍         | 3/67 [00:00<00:02, 24.42it/s]

labeling predictions...


  0%|          | 0/67 [00:00<?, ?it/s]

computing scores... 


...

labeling predictions...


  0%|          | 0/67 [00:00<?, ?it/s]

computing scores... 


100%|██████████| 67/67 [00:16<00:00,  4.14it/s]

expected LB score(0.09): 0.4563637017332251


In [ ]:
#whats going on here????
#step by step
#pa=model.predict_area(val).reshape((-1,)+model_shape[:2])
#pb=model.predict_boundary(val).reshape((-1,)+model_shape[:2])
idx=12
plt.imshow(pa[idx].reshape(pa[idx].shape[:2]))
plt.show()

plt.imshow(pb[idx].reshape(pb[idx].shape[:2]))
plt.show()

#those are OK!


In [ ]:
import skimage.restoration
#import peakutils
th=0.5
idx=0
lab_pred=np.zeros_like(pa, dtype=np.uint)
local_maxi=np.zeros_like(pa, dtype=np.uint)
markers=np.zeros_like(pa, dtype=np.uint)
i=idx
#for i in tqdm.tqdm(range(pa.shape[0])):
if 1:
    #get at least one pixel in each nuclei that is not connected to another
    _pa=copy.deepcopy(pa[i])
    _pb=copy.deepcopy(pb[i])
    _lab=scipy.ndimage.label(_pa>th)[0]
    _dist=scipy.ndimage.distance_transform_edt(_pa>.5)
    #_pb[_pb<.02]=0
    #_pb=skimage.restoration.denoise_tv_chambolle(_pb,multichannel=False)
    #local_maxi[i]=skimage.feature.peak_local_max(1-_pb, indices=False,threshold_rel=.9 , labels=_lab, exclude_border=3,min_distance=1 )
    local_maxi[i]=skimage.morphology.h_maxima((1-_pb), h=.01)
    local_maxi[i][_pa<th]=0
    #local_maxi[i]=skimage.feature.corner_peaks(1-_pb, indices=False,threshold_rel=.9 , labels=_lab, exclude_border=3)
    #label these starts
    markers[i] = scipy.ndimage.label(local_maxi[i])[0]
    lab_pred[i] = skimage.morphology.watershed(_pb, markers[i], mask=_pa>th)
    #lab_pred[i][_pa<th]=0
    lab_pred[i]=skimage.segmentation.relabel_sequential(lab_pred[i])[0]


In [ ]:



plt.imshow(val.images[idx])
plt.show()
plt.imshow(_lab)
plt.show()
#plt.imshow(local_maxi[idx])
#plt.show()

#plt.imshow(markers[idx])
#plt.show()

plt.imshow(lab_pred[idx])
plt.show()


In [ ]:
val.features['iou_score']

In [ ]:
idx=0
fig=plt.figure(figsize=(18, 12), dpi= 150, facecolor='w', edgecolor='k')
val.show_image(idx)

plt.imshow(pa[idx].reshape(pa[idx].shape[:2]))
plt.show()

plt.imshow(pb[idx].reshape(pb[idx].shape[:2]))
plt.show()



## Prediction (on test data)
* load test data
* use model to predict masks
* prepare submission file

In [18]:
test=Images.Images("../input/stage1_test")

#load the image files (in original size)    
test.load_images()
print('making predictions...')
test.add_predictions(model,boundary_height=0.06)
#much faster, no scores to calculate


100%|██████████| 65/65 [00:00<00:00, 100.04it/s]


making predictions...
predicting area...
predicting boundaries...


  0%|          | 0/65 [00:00<?, ?it/s]/home/lienhard/.local/lib/python3.6/site-packages/skimage/morphology/extrema.py:111: FutureWarning:

Conversion of the second argument of issubdtype from `'half'` to `np.floating` is deprecated. In future, it will be treated as `np.float16 == np.dtype('half').type`.

/home/lienhard/.local/lib/python3.6/site-packages/skimage/segmentation/_join.py:118: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.

  5%|▍         | 3/65 [00:00<00:03, 19.23it/s]

labeling predictions...


100%|██████████| 65/65 [00:03<00:00, 18.52it/s]


In [19]:
submission_file='submission_' + model_name + '.csv'
test.write_submission(submission_file)



0

In [ ]:

##########
# scores #
##########
print('Scores for first test image:')
#unlabeled IOU
print('IoU: {}'.format(Images.iou(val.masks[0] > 0, unlab_pred[0]>0.5)))
#labled IOUs per nucleus
print('IoU per nucleus: {}'.format(Images.iou(val.masks[0], val.pred[0])))
#mean IOU
print('mean IoU: {}'.format(np.mean(Images.iou(val.masks[0], val.pred[0]))))
# fraction of nuclei > th
print('fraction of nuclei has IoU > 0.5: {}'.format(Images.iou_score(val.masks[0], val.pred[0], th=[.5])))
print('fraction of nuclei has IoU > 0.95: {}'.format(Images.iou_score(val.masks[0], val.pred[0], th=[.95])))
# fraction of nuclei > th average over range of thresholds
print('IoU score (over range of thresholds): {}'.format(Images.iou_score(val.masks[0], val.pred[0])))

val.show_image(0)